<a href="https://colab.research.google.com/github/SamuelXJames/Auto-Music-Seperation/blob/main/preprocessing/dataHandler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os, sys, math, glob, ntpath,time
import tensorflow as tf
import timeit

class dataHandler:

  def __init__(self):
    
    
    return None
  
  
  def listFiles(self,tfr_dir,partion = None):
    files = tf.data.Dataset.list_files(os.path.join(tfr_dir,'*.tfrec'))
    
    if partion:
      files = files.batch(partion)
      files = list(files.as_numpy_iterator())
  
    return files
  
  
  @tf.function
  def get_tfrecord(self,GCS_File):
    dataset = tf.data.TFRecordDataset(GCS_File,compression_type='ZLIB', 
                                      num_parallel_reads = tf.data.AUTOTUNE)
    #dataset = dataset.batch(32)
    return dataset

  @tf.function
  def read_tfrecord(self,example):
    #Write this info into the filename
    nb_images = 1000
    HR_IMG_SHAPE = [84,84]
    LR_IMG_SHAPE = [21,21]
    NUM_CHANNELS = 3
  # Create a dictionary describing the features.
    features = {
        'HR': tf.io.FixedLenFeature([], tf.string),
        'HR_label': tf.io.FixedLenFeature([], tf.string),
        'LR': tf.io.FixedLenFeature([], tf.string),
        'LR_label': tf.io.FixedLenFeature([], tf.string)
    }
    
    example = tf.io.parse_example(example, features)

    HR = tf.map_fn(tf.io.decode_png, example['HR'], fn_output_signature=tf.uint8)
    HR = tf.map_fn(lambda HR: tf.reshape(HR, [HR_IMG_SHAPE[0],HR_IMG_SHAPE[1], NUM_CHANNELS]), HR)

    LR = tf.map_fn(tf.io.decode_png, example['LR'], fn_output_signature=tf.uint8)
    LR = tf.map_fn(lambda LR: tf.reshape(LR, [LR_IMG_SHAPE[0],LR_IMG_SHAPE[1], NUM_CHANNELS]), LR)
    
    HR_label = example['HR_label']
    LR_label = example['LR_label']
    
    return HR,HR_label,LR,LR_label
  
  def build_dataset(self,dataset, batch_size):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False

    #If the input is an array of file names, convert it to a dataset
    if type(dataset) == np.ndarray:
      dataset = tf.data.Dataset.from_tensor_slices(dataset)  
    

    dataset = dataset.interleave(self.get_tfrecord,
                                 block_length = 1,
                                 cycle_length = tf.data.AUTOTUNE,
                                 num_parallel_calls = tf.data.AUTOTUNE)
    
    dataset = dataset.batch(batch_size, drop_remainder = True)
  
    dataset = dataset.map(self.read_tfrecord,num_parallel_calls = tf.data.AUTOTUNE)
    
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

  
  

In [ ]:
#EXAMPLE
# dh = dataHandler()
# files = dh.listFiles('', partion=3)
# for partion in files:
#   ds = dh.build_dataset(partition, 32)
#   for (HR,HR_label,LR,LR_label) in ds.take(-1):
#       print('HR Shape: ' + str(np.shape(HR)) + ' LR Shape: ' + str(np.shape(LR)))



In [ ]:
#Different Methods to get batch
# ds = ds.cache() #if aplicable
# def a1():
#   count = 0 
#   for i,(HR,HR_label,LR,LR_label)  in enumerate(ds):
#     count = count + 1
#   return count

# def a2():
#   count = 0
#   for i in range(63):
#     count = count + 1
#   return count
# #tfds.core.benchmark

# def a3():
#   count = 0
#   for i in range(63):
#     HR,HR_label,LR,LR_label = next(iter(ds))
#     count = count + 1
#   return count

# def a4():
#   count = 0
#   for element in ds.as_numpy_iterator():
#     count = count + 1
#   return count

# def a5():
#   count = 0
#   for element in iter(ds):
#     count = count + 1
#   return count

# def a6():
#   count = 0
#   for element in ds.take(-1):
#     count = count + 1
#   return count

# #count = a4()
# timeit.timeit(a6, number=10)

0.46835634999843023

In [ ]:
#a1 = 9.496949963000588
#a3 = 90s?
#a4 = 9.607164383000054
#a5 = 9.526504041999942
#a6 = 9.462812317000498

9.496949963000588